In [1]:
from text_processing import get_text_map
from os import listdir
from os.path import join
import random
import json
from collections import OrderedDict
import numpy as np
import pandas as pd

In [4]:
texts_lenta = pd.read_csv("./colloc/music_lenta.csv")

In [3]:
texts_index = [0,
17,
26,
32,
98,
121,
130,
133,
138,
182,
207,
210,
249,
264,
289]

In [2]:
texts_index = [0,
17,
26,
32]

In [5]:
text_maps_json_dict = {}
for tex_ind in texts_index:
    text_map = get_text_map(texts_lenta.iloc[tex_ind]['text'], raw_text_input=True)
    text_maps_json_dict[tex_ind] = text_map

In [11]:
text_maps_json_dict[32]['overall_colloc_text']

[{0: [(0, 'дэвид'),
   (2, 'всетаки'),
   (3, 'мочь'),
   (4, 'отправляться'),
   (5, 'в'),
   (6, 'тур'),
   (7, 'в'),
   (8, 'поддержка'),
   (9, 'свой'),
   (10, 'новый'),
   (12, 'the')]},
 {1: [(0, 'о это он'),
   (3, 'жена'),
   (6, 'полный'),
   (7, 'имя'),
   (11, 'сообщать'),
   (13, 'издание')]},
 {2: [(0, 'печатный'), (1, 'версия'), (2, 'журнал'), (3, 'цитировать')]},
 {3: [(0, 'в'),
   (1, 'разговор'),
   (2, 'о'),
   (3, 'план'),
   (4, 'на'),
   (5, 'ближайший год'),
   (8, 'сказать'),
   (9, 'о то что'),
   (12, 'семья'),
   (14, 'не мочь'),
   (16, 'сопровождать'),
   (17, 'он в'),
   (19, 'планировать'),
   (20, 'поездка')]},
 {4: [(0, 'у мы'),
   (3, 'дочь'),
   (4, 'школьница'),
   (5, 'и'),
   (6, 'наш'),
   (7, 'график'),
   (8, 'зависеть от'),
   (10, 'она'),
   (11, 'расписание')]},
 {5: [(0, 'надо'),
   (1, 'быть'),
   (3, 'дэвид'),
   (4, 'безусловно'),
   (5, 'но в'),
   (7, 'тур'),
   (8, 'мы'),
   (9, 'с он'),
   (11, 'не'),
   (12, 'отправиться'),
   (13, '

In [6]:
question_text_sent_collocind_dict = {"0":{'0':[9,11],'2':[26,35],'5':[13,27]},
                                     "17":{'0':[7,18],'2':[8,19],'4':[2,16],'8':[7,25],'10':[14,24],'12':[9,24]},
                                     "26":{'0':[6,19],'5':[5,11],'7':[0,6],'13':[13,27]},
                                     "32":{'3':[9,20],'5':[5,17],'14':[6,17],'18':[3,19]}}

answers_dict = {"0":{'0': True,'2':True,'5':False},
                 "17":{'0':True,'2':False,'4':True,'8':False,'10':False,'12':True},
                 "26":{'0':True,'5':True,'7':True,'13':False},
                 "32":{'3':True,'5':False,'14':True,'18':False}}

In [16]:
effected_collocations_start_indexes_list = [i for i in range(effected_collocations_start_indexes_list[0],effected_collocations_start_indexes_list[1]+1)]

[1, 2, 3, 4]

In [7]:
class user_vector:
    def __init__(self,debug = False):
        self.debug = debug
        self.vocab_features =[]
        self.sentence_features =[]
        #coreference_items, negation_items, sent_special_pos, dependencies_length, Y (answer)                   
        self.text_fearues = [] #OrderedDict([("lix",[]),("ttr",[])])
        self.answers_count = OrderedDict([("correct_answers",0),("incorrect_answers",0)])
        self.collocations_list = []
    def start_new_text(self):
        self.answers_count['correct_answers'] = 0
        self.answers_count['incorrect_answers'] = 0
        if self.debug:print("answers count has been reset", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
    
    def end_text(self, text_map):
        if self.debug:
            print("\n========")
            print("SUM UP TEXT VALUES")
            print("========\n")
        correct_answers_rate = round(self.answers_count['correct_answers'] / (self.answers_count['correct_answers'] 
                                                                              + self.answers_count['incorrect_answers']),2)
        current_text_features = []
        if self.debug: 
            print("answers_count", self.answers_count)
        current_text_features.append(text_map['lix'])
        current_text_features.append(text_map['ttr'])
        #current_text_features.extend(text_map['vocab_properties'])
        current_text_features.extend(text_map['sent_properties'])
        current_text_features.append(correct_answers_rate)
        self.text_fearues.append(current_text_features)
        if self.debug: print("TEXT FEATURES",self.text_fearues)
    
    def update_vector_with_answer_sentence(self, sentence_map, effected_collocations_start_indexes_list, correct_answer):
        #update setnence and text features
        if self.debug:
            print("\n===NEW REPLY CALCULATION====")
            print("\n========")
            print("ADDING SENTENCE RESULTS")
            print("========\n")
            
        #update setnence and text features
        if correct_answer == True:
            answer_value = 1
            self.answers_count['correct_answers'] += 1
            if self.debug: print("Answer for this question is correct")
        else:
            answer_value = 0
            self.answers_count['incorrect_answers'] += 1
            if self.debug: print("Answer for this question is incorrect")
        if self.debug:print("check answers count", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
        current_sentence_features = []
        current_sentence_features.append(sentence_map['spec_sentence_features']['negation'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['coreference'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['vozvr_verb'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['prich'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['deepr'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['case_complexity'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['mean_depend_length'])
        
        #current_sentence_features.extend(sentence_map['average_vocabulary'])
        current_sentence_features.append(answer_value)#target variable
        self.sentence_features.append(current_sentence_features)
        
        if self.debug: print("SENTENCE FEATURES", current_sentence_features)
        
        if self.debug:
            print("\n========")
            print("ADDING VOCABULARY RESULTS")
            print("========\n") 
            
        for word_w in sentence_map['collocation_vectors_list']:
            if self.debug:print(word_w[1][0])
            effected_collocations_start_indexes_list_ranged = [i for i in range(effected_collocations_start_indexes_list[0],effected_collocations_start_indexes_list[1]+1)]
            if word_w[0] in effected_collocations_start_indexes_list_ranged:
                current_word_features = []
                #current_word_features.append(str("local_freq_") + str(word_w[1][1]))
                current_word_features.append(word_w[1][1])
                #current_word_features.append(str("global_freq_mpi_ln_") + glob_freq_log)
                current_word_features.append(word_w[1][2])
                current_word_features.extend(word_w[2][0])
                current_word_features.append(answer_value)
                self.collocations_list.append(word_w[1][0])
                #print(current_word_features)
                self.vocab_features.append(current_word_features)
    def export_user_db(self, learner_id):
                
        words_db = np.array([np.array(word) for word in self.vocab_features])
        word_db_path = learner_id + '_word_db.csv'
        np.savetxt(word_db_path, words_db, delimiter=',') 
        
        sentence_db = np.array([np.array(sent) for sent in self.sentence_features])
        sentence_db_path = learner_id + '_sentence_db.csv'
        np.savetxt(sentence_db_path, sentence_db, delimiter=',') 
        
        text_db = np.array([np.array(text) for text in self.text_fearues])
        text_db_path = learner_id + '_text_db.csv'
        np.savetxt(text_db_path, text_db, delimiter=',') 
        
        with open("colloc_lit.txt", "w", encoding = "utf-8") as f:
            for colloc in self.collocations_list:
                f.write(colloc + '\n')

In [19]:
text_maps_json_dict.keys()

dict_keys([0, 17, 26, 32, 98])

In [31]:
question_text_sent_collocind_dict['0']['0']

[9, 11]

In [36]:
text_maps_json_dict.keys()

dict_keys([0, 17, 26, 32])

In [8]:
DEBUG = True
user = user_vector(debug = DEBUG)
for text in text_maps_json_dict.keys():
    print(text)
    text_map = text_maps_json_dict[text]
    user.start_new_text()
    for question_sentence_index in question_text_sent_collocind_dict[str(text)].keys():
        
        user.update_vector_with_answer_sentence(text_map['sentences_map'][int(question_sentence_index)], 
                                        effected_collocations_start_indexes_list = question_text_sent_collocind_dict[str(text)][str(question_sentence_index)],
                                        correct_answer = answers_dict[str(text)][str(question_sentence_index)])

    user.end_text(text_map)

0
answers count has been reset 0 0

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is correct
check answers count 1 0
SENTENCE FEATURES [0, 0, 0, 0, 0, 0, 0.2090909090909091, 1]

ADDING VOCABULARY RESULTS

расход на
проведение
музыкальный
конкурс
2017
в
киев
оказываться
предельно
высокий

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is correct
check answers count 2 0
SENTENCE FEATURES [0.018518518518518517, 0, 0.021353453732970453, 0, 0, 0.042706907465940906, 0.34716981132075475, 1]

ADDING VOCABULARY RESULTS

украинец
гонорар
артист
и
ведущий
шоу
то что
почти
весь
тендер
проводить в
сжимать
срок
по
процедура
а также
явно
лишний
трата
в страна
где
идти
война
речь идти о
вооружать
конфликт
в
примечание лентыру
а
большой часть
население
не хватать
деньги
даже
на
оплата
говорить в
заметка

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is incorrect
check answers count 2 1
SENTENCE FEATURES

In [9]:
user.export_user_db("littel_musician")  